In [ ]:
import os
import sys

import copy
import numpy as np
from astropy.cosmology import Planck18

PACKAGE_PARENT = '..'
SCRIPT_DIR = os.path.dirname(os.path.realpath(os.path.join(os.getcwd())))
sys.path.append(SCRIPT_DIR)
import gwfast.gwfastGlobals as glob
import gwfast 
from gwfast.waveforms import IMRPhenomD_NRTidalv2
from gwfast.signal import GWSignal
from gwfast.network import DetNet
from fisherTools import CovMatr, compute_localization_region, check_covariance, fixParams
from gwfastUtils import GPSt_to_LMST

In [ ]:
print(glob.dirName,SCRIPT_DIR)

In [ ]:
keys = ('Mc','eta','dL','theta','phi','iota','psi','tcoal','Phicoal','chi1x','chi2x',
        'chi1y','chi2y','chi1z','chi2z','LambdaTilde','deltaLambda','ecc')
mydata=dict.fromkeys(keys)

In [ ]:
print(mydata.keys())

In [ ]:
alldetectors = copy.deepcopy(glob.detectors)
print('All available detectors are: '+str(list(alldetectors.keys())))

# select only LIGO and Virgo
LVdetectors = {det:alldetectors[det] for det in ['L1', 'H1', 'Virgo']}
print('Using detectors '+str(list(LVdetectors.keys())))

In [ ]:
LVdetectors

In [ ]:
# We use the O2 psds
LVdetectors['L1']['psd_path'] = os.path.join(glob.detPath, 'LVC_O1O2O3', '2017-08-06_DCH_C02_L1_O2_Sensitivity_strain_asd.txt')
LVdetectors['H1']['psd_path'] = os.path.join(glob.detPath, 'LVC_O1O2O3', '2017-06-10_DCH_C02_H1_O2_Sensitivity_strain_asd.txt')
LVdetectors['Virgo']['psd_path'] = os.path.join(glob.detPath, 'LVC_O1O2O3', 'Hrec_hoft_V1O2Repro2A_16384Hz.txt')


In [ ]:
glob.detPath

In [ ]:
myLVSignals = {}

for d in LVdetectors.keys():

    myLVSignals[d] = GWSignal(IMRPhenomD_NRTidalv2(),
                psd_path=LVdetectors[d]['psd_path'],
                detector_shape = LVdetectors[d]['shape'],
                det_lat= LVdetectors[d]['lat'],
                det_long=LVdetectors[d]['long'],
                det_xax=LVdetectors[d]['xax'],
                verbose=True,
                useEarthMotion = False,
                fmin=10.,
                IntTablePath=None)

myLVNet = DetNet(myLVSignals)

In [ ]:


# Median values of the posterior samples for all the parameters,
# except psi and the coalescence phase that are set to 0

z = np.array([0.00980])
tGPS = np.array([1187008882.4])

GW170817 = {'Mc':np.array([1.1859])*(1.+z),
            'dL':Planck18.luminosity_distance(z).value/1000.,
            'theta':np.array([np.pi/2. + 0.4080839999999999]),
            'phi':np.array([3.4461599999999994]),
            'iota':np.array([2.545065595974997]),
            'psi':np.array([0.]),
            'tcoal':GPSt_to_LMST(tGPS, lat=0., long=0.), # GMST is LMST computed at long = 0°
            'eta':np.array([0.24786618323504223]),
            'Phicoal':np.array([0.]),
            'chi1z':np.array([0.005136138323169717]),
            'chi2z':np.array([0.003235146993487445]),
            'Lambda1':np.array([368.17802383555687]),
            'Lambda2':np.array([586.5487031450857])
           }

print('Parameters for GW170817 are:')
GW170817

In [ ]:
SNR = myLVNet.SNR(GW170817)
print('SNR for GW170817 is %.2f to compare with 33'%SNR[0])

In [ ]:
totF = myLVNet.FisherMatr(GW170817)
print('The computed Fisher matrix has shape %s'%str(totF.shape))

In [ ]:
totF

In [ ]:
# Check e.g. that the (dL,dL) element corresponds to (SNR/dL)^2
ParNums = IMRPhenomD_NRTidalv2().ParNums
dL_Num = ParNums['dL']
print('The relative difference is %.2e !'%((1 - totF[ParNums['dL'],ParNums['dL'],:]/(SNR/GW170817['dL'])**2)[0]))

In [ ]:
# Check e.g. that the (dL,dL) element corresponds to (SNR/dL)^2
ParNums = IMRPhenomD_NRTidalv2().ParNums
dL_Num = ParNums['dL']
print('The relative difference is %.2e !'%((1 - totF[ParNums['dL'],ParNums['dL'],:]/(SNR/GW170817['dL'])**2)[0]))

In [ ]:
totCov, inversion_err = CovMatr(totF)

In [ ]:
totCov

In [ ]:
_ = check_covariance(totF, totCov)

'Mc','eta','dL','theta','phi','iota','psi','tcoal','Phicoal','chi1z','chi2z','LambdaTilde','deltaLambda'}

In [ ]:
ParNums = IMRPhenomD_NRTidalv2().ParNums
tofix_ParNums=[e for e in ParNums if e not in ('dL', 'phi','theta')]
#newFish, newPars = fixParams(totF, ParNums, ['deltaLambda'])
newFish, newPars = fixParams(totF, ParNums, tofix_ParNums)
print('Now the Fisher matrix has shape %s'%str(newFish.shape))

newCov, new_inversion_err = CovMatr(newFish)

_ = check_covariance(newFish, newCov)

In [ ]:
skyArea = compute_localization_region(newCov, newPars, GW170817['theta'])
print('The estimated sky location is %.1f deg^2, to compare with 16 deg^2'%skyArea)

In [ ]:
save_path='/home/rciancarella/testgwfast.h5'
gwfast.gwfastUtils.save_data(save_path, GW170817)
gwfast.gwfastUtils.save_detectors('/home/rciancarella/net.json',LVdetectors)

In [ ]:
print(newCov)

# Play Around

In [ ]:
# Configure ET and the PSD
ETdet = {'ET': copy.deepcopy(glob.detectors).pop('ETS') }
print(ETdet)
ETdet['ET']['psd_path'] = os.path.join(glob.detPath, 'ET-0000A-18.txt')

In [ ]:
from gwfast.waveforms import IMRPhenomD
from gwfast.waveforms import IMRPhenomHM
mySignalsET = {}

for d in ETdet.keys():
    #print(d)
    mySignalsET[d] = GWSignal((IMRPhenomD()),
                psd_path= ETdet[d]['psd_path'],
                detector_shape = ETdet[d]['shape'],
                det_lat= ETdet[d]['lat'],
                det_long=ETdet[d]['long'],
                det_xax=ETdet[d]['xax'],
                verbose=True,
                useEarthMotion = False,
                fmin=2.,
                IntTablePath=None)

myET = DetNet(mySignalsET)

In [ ]:
import pandas as pd
from astropy.table import Table
from astropy.cosmology import FlatLambdaCDM

In [ ]:
CAT_FOLDER='/home/rciancarella/DarkSirensStat/DSCatalogueCreator/'
SCRIPT_FOLDER='/home/rciancarella/DarkSirensStat/GWfast/'
os.chdir(CAT_FOLDER)

In [ ]:
dat = Table.read('EVA02_detected12_fiveyear.fits', format='fits')
EVA= dat.to_pandas()#all good, is an only text fits
os.chdir(SCRIPT_FOLDER)

In [ ]:
EVA.head()

Raccordo tra i dati 
GW170817 = {'Mc':np.array([1.1859])*(1.+z),
            'dL':Planck18.luminosity_distance(z).value/1000.,
            'theta':np.array([np.pi/2. + 0.4080839999999999]),
            'phi':np.array([3.4461599999999994]),
            'iota':np.array([2.545065595974997]),
            'psi':np.array([0.]),
            'tcoal':GPSt_to_LMST(tGPS, lat=0., long=0.), # GMST is LMST computed at long = 0°
            'eta':np.array([0.24786618323504223]),
            'Phicoal':np.array([0.]),
            'chi1z':np.array([0.005136138323169717]),
            'chi2z':np.array([0.003235146993487445]),
            'Lambda1':np.array([368.17802383555687]),
            'Lambda2':np.array([586.5487031450857])
           }

In [ ]:
ParNums = IMRPhenomD().ParNums
print('In totale sono {} parametri da inserire\ne sono {}'.format(len(ParNums),ParNums))

In [ ]:
H0GLOB= 67#67.9 #69
Om0GLOB=0.319
Xi0Glob =1.
cosmoeuclid = FlatLambdaCDM(H0=H0GLOB, Om0=Om0GLOB)

In [ ]:
def chirp_mass(m1,m2):
    num=(m1*m2)**(3/5)
    denom=(m1+m2)**(1/5)
    return num/denom
def phi_from_cart(x,y):
    if(x>0):
        phi=np.arctan(y/x)
    if ((x<0) and (y>=0)):
        phi=np.arctan(y/x)+np.pi
    if ((x<0) and (y<0)):
        phi=np.arctan(y/x)-np.pi
    return phi
def theta_from_cart(x,y,z):
    den=np.sqrt(x**2+y**2+z**2)
    theta=np.arccos(z/den)
    return theta

In [ ]:
allm1=np.asarray(EVA['M1'])
allm2=np.asarray(EVA['M2'])
allMc=chirp_mass(allm1,allm2)
allq=np.asarray(EVA['q'])
alleta=allq/(1+allq**2)
allcos=np.asarray(EVA['cos_orbital_incl'])
alliota=np.arccos(allcos)
allpsi=np.asarray(EVA['psi'])/2
tGPS = np.array([1187008882.4])#arbitrario
allz=np.asarray(EVA['DS_redshift'])
alldsx=np.asarray(EVA['DS_x'])
alldsy=np.asarray(EVA['DS_y'])
alldsz=np.asarray(EVA['DS_z'])
allphi=np.zeros(len(alldsx))
alltheta=np.zeros(len(alldsx))
for i in range(len(alldsx)):
    allphi[i]=phi_from_cart(alldsx[i],alldsy[i])
    alltheta[i]=theta_from_cart(alldsx[i],alldsy[i],alldsz[i])
alldl=cosmoeuclid.luminosity_distance(allz).value/1000.


tcoal=np.asarray(GPSt_to_LMST(tGPS, lat=0., long=0.))

In [ ]:
len(alliota[alliota<0])

In [ ]:
quanti=int(1)
Allevents = {'Mc':1*allMc[0:quanti]*(1+allz)[0:quanti],
            'dL':alldl[0:quanti],
            'theta':alltheta[0:quanti],
            'phi':allphi[0:quanti],
            'iota':alliota[0:quanti],
            'psi':allpsi[0:quanti],
            'tcoal':1*tcoal*np.ones(len(allMc))[0:quanti], # GMST is LMST computed at long = 0°
            'eta':alleta[0:quanti],
            'Phicoal':np.zeros(len(allMc))[0:quanti],
            'chi1z':0.0001*np.ones(len(allMc))[0:quanti],
            'chi2z':0.*np.ones(len(allMc))[0:quanti]
            ,#'chi2z':np.zeros(len(allMc))[0:1]
           }

In [ ]:
save_path='/home/rciancarella/DarkSirensStat/GWfast/myfirstevent.h5'
gwfast.gwfastUtils.save_data(save_path, Allevents)

In [ ]:
Allevents=gwfast.gwfastUtils.load_population('myfirstevent.h5')

In [ ]:
Allevents

In [ ]:
SNR_ET = myET.SNR(Allevents)
#print('SNR is %.2f '%SNR)
print(SNR_ET)

In [ ]:
totFET = myET.FisherMatr(Allevents)
print('The computed Fisher matrix has shape %s'%str(totFET.shape))

In [ ]:
#print('Fisher Matrix\n{}'.format(totFET))
n_array = totFET[:,:,0]
  
# Displaying the Matrix 
#print("Numpy Matrix is:") 
#print(n_array) 
  
# calculating the determinant of matrix 
det = np.linalg.det(n_array) 
  
print("\nDeterminant of given matrix:") 
print(det) 

In [ ]:
totFET

In [ ]:
from gwfast import fisherTools
gwfast.fisherTools.CheckFisher(totFET)

In [ ]:
gwfast.fisherTools.CovMatr(totFET)

In [ ]:
totCov_ET, inversion_err_ET = CovMatr(totFET)

In [ ]:
totCov_ET

In [ ]:
os.chdir(SCRIPT_FOLDER+'results/')

In [ ]:
loaded_fish=np.load('fishers_0_to_1.npy')

In [ ]:
np.shape(loaded_fish)

Marginalisation------in the doc there is no marginalisation

In [ ]:
events=gwfast.gwfastUtils.load_population('myfirstevent.h5')

In [ ]:
print(events)

In [ ]:
FIM=myET.FisherMatr(events)

In [ ]:
#fixing Parameters
#print(events.keys())
tofix_ParNums = list(IMRPhenomD().ParNums.keys())
tofix_ParNums = [e for e in tofix_ParNums if e not in ('dL', 'phi','theta')]
fixedFIM, newPars=gwfast.fisherTools.fixParams(totFET,IMRPhenomD().ParNums,tofix_ParNums)

In [ ]:
print(newPars)

In [ ]:
mycov,err=gwfast.fisherTools.CovMatr(fixedFIM)

In [ ]:
print(mycov)

In [ ]:
COV_SAVE_PATH='/home/rciancarella/DarkSirensStat/ProxyGenerator/'

In [ ]:
name='covar_test'

In [ ]:
np.save(COV_SAVE_PATH+name,totCov_ET)

In [ ]:
skyArea = compute_localization_region(totCov_ET, newPars, Allevents['theta'])
print('The estimated sky location is %.1f deg^2'%skyArea)

In [ ]:
Allevents['theta'][0]

In [ ]:
# imports
from importlib import reload
import os
from pkg_resources import resource_filename
import numpy as np

import healpy as hp
import pandas

from astropy.io import fits
from astropy.table import Table
from astropy.coordinates import SkyCoord
#from astropath import bayesian
from astropath import localization
from astropath import path

In [ ]:
lfile = os.path.join(resource_filename('astropath', 'data'), 'gw_examples',
                     'GW170817_skymap.fits.gz')

# To ProxyGenerator

In [1]:
import os
import sys

import copy
import numpy as np
from astropy.cosmology import Planck18

import pandas as pd
from astropy.table import Table
from astropy.cosmology import FlatLambdaCDM

PACKAGE_PARENT = '..'
SCRIPT_DIR = os.path.dirname(os.path.realpath(os.path.join(os.getcwd())))
sys.path.append(SCRIPT_DIR)
import gwfast.gwfastGlobals as glob
import gwfast 
from gwfast.waveforms import IMRPhenomD_NRTidalv2
from gwfast.signal import GWSignal
from gwfast.network import DetNet
from fisherTools import CovMatr, compute_localization_region, check_covariance, fixParams
from gwfastUtils import GPSt_to_LMST

TEOBResumS is not installed, only the GWFAST waveform models are available, namely: TaylorF2, IMRPhenomD, IMRPhenomD_NRTidalv2, IMRPhenomHM and IMRPhenomNSBH


In [2]:
H0GLOB= 67#67.9 #69
Om0GLOB=0.319
Xi0Glob =1.
cosmoeuclid = FlatLambdaCDM(H0=H0GLOB, Om0=Om0GLOB)

In [3]:
def chirp_mass(m1,m2):
    num=(m1*m2)**(3/5)
    denom=(m1+m2)**(1/5)
    return num/denom
def phi_from_cart(x,y):
    if(x>0):
        phi=np.arctan(y/x)
    if ((x<0) and (y>=0)):
        phi=np.arctan(y/x)+np.pi
    if ((x<0) and (y<0)):
        phi=np.arctan(y/x)-np.pi
    return phi
def theta_from_cart(x,y,z):
    den=np.sqrt(x**2+y**2+z**2)
    theta=np.arccos(z/den)
    return theta

In [4]:
COV_SAVE_PATH='/home/rciancarella/DarkSirensStat/ProxyGenerator/'
CAT_FOLDER='/home/rciancarella/DarkSirensStat/DSCatalogueCreator/'
SCRIPT_FOLDER='/home/rciancarella/DarkSirensStat/GWfast/'

In [5]:
ETdet = {'ET': copy.deepcopy(glob.detectors).pop('ETS') }
print(ETdet)
ETdet['ET']['psd_path'] = os.path.join(glob.detPath, 'ET-0000A-18.txt')

{'ET': {'lat': 40.516666666666666, 'long': 9.416666666666666, 'xax': 0.0, 'shape': 'T'}}


In [6]:
from gwfast.waveforms import IMRPhenomD
from gwfast.waveforms import IMRPhenomHM
mySignalsET = {}

for d in ETdet.keys():
    #print(d)
    mySignalsET[d] = GWSignal((IMRPhenomD()),
                psd_path= ETdet[d]['psd_path'],
                detector_shape = ETdet[d]['shape'],
                det_lat= ETdet[d]['lat'],
                det_long=ETdet[d]['long'],
                det_xax=ETdet[d]['xax'],
                verbose=True,
                useEarthMotion = False,
                fmin=2.,
                IntTablePath=None)

myET = DetNet(mySignalsET)

Using ASD from file /home/rciancarella/.local/lib/python3.10/site-packages/psds/ET-0000A-18.txt 
Initializing jax...
Jax local device count: 8
Jax  device count: 8


In [7]:
os.chdir(CAT_FOLDER)
dat = Table.read('EVA02_detected12_fiveyear.fits', format='fits')
EVA= dat.to_pandas()#all good, is an only text fits
os.chdir(SCRIPT_FOLDER)

In [8]:
EVA.head(10)

,Host_x,Host_y,Host_z,Host_redshift,DS_x,DS_y,DS_z,DS_redshift,cos_orbital_incl,M1,M2,q,psi,Aproximant,SNR
0,651.933995,402.188065,2806.954546,0.808803,651.933995,402.188065,2806.954546,0.808803,0.793474,10.569518,5.143687,0.486653,2.550917,b'IMRPhenomD',96.279798
1,1901.947841,477.181005,2147.676656,0.808301,1901.947841,477.181005,2147.676656,0.808301,-0.615400,26.536605,5.515256,0.207836,3.274064,b'IMRPhenomD',81.336203
2,792.327061,2056.292634,1873.462461,0.802654,796.891722,2051.491963,1868.765822,0.800800,0.228812,38.501322,29.412304,0.763930,3.479153,b'IMRPhenomD',114.008069
3,2566.548362,58.913997,1323.989640,0.801273,2571.261103,58.839892,1319.617234,0.802053,0.694977,14.085194,1.642785,0.116632,4.078907,b'IMRPhenomD',29.901415
4,1165.316454,2573.915761,575.600923,0.799466,1165.316454,2573.915761,575.600923,0.799466,-0.838562,5.332573,2.141039,0.401502,2.813217,b'IMRPhenomD',22.195059
5,1690.215609,1708.191465,1620.095871,0.804719,1691.545189,1707.465640,1623.334204,0.805490,-0.871574,13.908275,5.170403,0.371750,2.143557,b'IMRPhenomD',72.726116
6,1794.658786,1246.884474,1882.023962,0.799664,1794.954494,1253.441632,1883.973892,0.801195,0.627667,48.437785,5.166575,0.106664,0.451809,b'IMRPhenomD',80.957316
7,924.807420,1080.356518,2565.594027,0.817343,924.807420,1080.356518,2565.594027,0.817343,-0.910816,16.017793,2.260788,0.141142,3.311528,b'IMRPhenomD',74.167630
8,1.287816,620.860268,2850.996968,0.811749,1.287816,620.860268,2850.996968,0.811749,0.961688,9.920316,1.330151,0.134083,2.643927,b'IMRPhenomD',59.470214
9,1532.263730,2409.690379,418.617545,0.800414,1537.213387,2410.695433,414.988671,0.801464,-0.554800,5.691874,1.697490,0.298230,3.966257,b'IMRPhenomD',13.890744


In [9]:
ParNums = IMRPhenomD().ParNums
print('In totale sono {} parametri da inserire\ne sono {}'.format(len(ParNums),ParNums))

In totale sono 11 parametri da inserire
e sono {'Mc': 0, 'eta': 1, 'dL': 2, 'theta': 3, 'phi': 4, 'iota': 5, 'psi': 6, 'tcoal': 7, 'Phicoal': 8, 'chi1z': 9, 'chi2z': 10}


In [10]:
allm1=np.asarray(EVA['M1'])
allm2=np.asarray(EVA['M2'])
allMc=chirp_mass(allm1,allm2)
allq=np.asarray(EVA['q'])
alleta=allq/(1+allq**2)
allcos=np.asarray(EVA['cos_orbital_incl'])
alliota=np.arccos(allcos)
allpsi=np.asarray(EVA['psi'])/2
tGPS = np.array([1187008882.4])#arbitrario
allz=np.asarray(EVA['DS_redshift'])
alldsx=np.asarray(EVA['DS_x'])
alldsy=np.asarray(EVA['DS_y'])
alldsz=np.asarray(EVA['DS_z'])
allphi=np.zeros(len(alldsx))
alltheta=np.zeros(len(alldsx))
for i in range(len(alldsx)):
    allphi[i]=phi_from_cart(alldsx[i],alldsy[i])
    alltheta[i]=theta_from_cart(alldsx[i],alldsy[i],alldsz[i])
alldl=cosmoeuclid.luminosity_distance(allz).value/1000.


tcoal=np.asarray(GPSt_to_LMST(tGPS, lat=0., long=0.))

In [11]:
quanti=int(1)
Allevents = {'Mc':1*allMc[0:quanti]*(1+allz)[0:quanti],
            'dL':alldl[0:quanti],
            'theta':alltheta[0:quanti],
            'phi':allphi[0:quanti],
            'iota':alliota[0:quanti],
            'psi':allpsi[0:quanti],
            'tcoal':1*tcoal*np.ones(len(allMc))[0:quanti], # GMST is LMST computed at long = 0°
            'eta':alleta[0:quanti],
            'Phicoal':0.001*np.ones(len(allMc))[0:quanti],
            'chi1z':0.*np.ones(len(allMc))[0:quanti],
            'chi2z':0.*np.ones(len(allMc))[0:quanti]
            ,#'chi2z':np.zeros(len(allMc))[0:1]
           }

In [12]:
#save_path='/home/rciancarella/DarkSirensStat/GWfast/myfirstevent.h5'
gwfast.gwfastUtils.save_data(COV_SAVE_PATH+'myfirstevent.h5', Allevents)

Saving to /home/rciancarella/DarkSirensStat/ProxyGenerator/myfirstevent.h5 


In [13]:
totFET = myET.FisherMatr(Allevents)
print('The computed Fisher matrix has shape %s'%str(totFET.shape))

Computing Fisher for ET...
Computing derivatives...
Filling matrix for arm 1...
Computing derivatives...
Filling matrix for arm 2...
Filling matrix for arm 3...
Done.
The computed Fisher matrix has shape (11, 11, 1)


In [23]:
tofix_ParNums = list(IMRPhenomD().ParNums.keys())
tofix_ParNums = [e for e in tofix_ParNums if e in  ('chi1z','chi2z','tcoal','Phicoal')]
fixedFIM, newPars=fixParams(totFET,IMRPhenomD().ParNums,tofix_ParNums)

NameError: name 'lines' is not defined

In [29]:
print(np.array(list(newPars.keys())))
keys=np.array(list(newPars.keys()))
print(type(keys))
import pickle
with open(COV_SAVE_PATH+'param.plk', 'wb') as file:
    pickle.dump(keys, file)

['Mc' 'eta' 'dL' 'theta' 'phi' 'iota' 'psi']
<class 'numpy.ndarray'>


In [19]:
totCov_ET, inversion_err_ET = CovMatr(fixedFIM)

In [20]:
check_covariance(fixedFIM,totCov_ET)

Inversion errors: [1.28721922e-10]
diagonal-1 = [array([ 1.17655885e-13,  8.14626144e-15, -5.49940952e-14, -3.85524945e-14,
       -2.22183383e-14, -2.66252949e-14, -2.20101715e-14], dtype=float128)]
Max off diagonal: [9.8339114629197865725e-11]

mask: where F*S(off-diagonal)>1e-10 (--> problematic if True off diagonal)
[array([[ True, False, False, False, False, False, False],
       [False,  True, False, False, False, False, False],
       [False, False,  True, False, False, False, False],
       [False, False, False,  True, False, False, False],
       [False, False, False, False,  True, False, False],
       [False, False, False, False, False,  True, False],
       [False, False, False, False, False, False,  True]])]


[array([[ 1.00000000e+00, -1.93872696e-14,  5.55007751e-18,
         -6.86625236e-16,  3.63478778e-17,  2.09450072e-18,
         -1.21213803e-16],
        [-2.29816166e-14,  1.00000000e+00,  1.22446553e-18,
         -4.98841420e-16,  1.77077320e-16, -1.48506051e-17,
         -3.49709411e-16],
        [ 9.83391146e-11,  1.93551841e-11,  1.00000000e+00,
         -8.91953178e-13,  3.25045546e-13, -3.91078229e-14,
          2.89546165e-13],
        [-4.89919216e-12, -8.38440428e-13,  5.91749922e-16,
          1.00000000e+00,  4.26915447e-15, -1.33127830e-15,
         -1.32723693e-14],
        [-4.00035560e-12, -7.80353560e-12,  6.14800908e-15,
         -9.50906021e-14,  1.00000000e+00,  5.52975634e-15,
         -4.42770820e-14],
        [-8.39719405e-11,  6.32027763e-12,  4.77799088e-15,
          1.38999923e-13,  7.48706652e-15,  1.00000000e+00,
         -1.45494727e-13],
        [-1.28721922e-10,  1.20934374e-11,  1.73240599e-15,
          2.40807374e-13,  2.89490654e-14, -3.92914867e-16

In [25]:
name='covar_test'

In [30]:
mymu=np.zeros(len(keys))
for i,j in enumerate(keys):
    mymu[i]=Allevents[j]
    print(j)
#mymu[0]=Allevents['dL']
#mymu[1]=Allevents['theta']
#mymu[2]=Allevents['phi']
#mymu[3]=Allevents['eta']
#mymu[4]=Allevents['Mc']
#mymu[5]=Allevents['iota']
#mymu[6]=Allevents['psi']
#keys=['dL', 'theta', 'phi', 'Mc', 'eta', 'iota', 'psi']

Mc
eta
dL
theta
phi
iota
psi


In [31]:
np.save(COV_SAVE_PATH+name,totCov_ET)
os.chdir(COV_SAVE_PATH)
#parname='param.txt'
#np.savetxt(parname,keys,fmt='%s')
np.savetxt('mean.txt',mymu)

In [28]:
SNR_ET = myET.SNR(Allevents)
#print('SNR is %.2f '%SNR)
print(SNR_ET)
skyArea = compute_localization_region(totCov_ET, newPars, Allevents['theta'])
print('The estimated sky location is %.1f deg^2'%skyArea)

[40.934412]
The estimated sky location is 7.9 deg^2
